In [4]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
ws.write_config(path='.azureml')
# ws = Workspace.get(name="quick-starts-ws-154239",
#                    resource_group="aml-quickstarts-154239",
#                    subscription_id="5a4ab2ba-6c51-4805-8155-58759ad589d8")
exp = Experiment(workspace=ws, name="udacity-project-1")


In [5]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-154263
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-154263


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
   cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
   print('Found existing cluster, use it.')
except ComputeTargetException:
   compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                          max_nodes=4)
   cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C': uniform(0,0.5),
        '--max_iter': choice(range(10,110,10)),
      }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = ".",entry_script = "train.py",compute_target = cpu_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est, 
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy', 
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs = 20,
                                     max_concurrent_runs =4)
                                    

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_0527fb3f-98be-4473-9bc3-32fdd153d953
Web View: https://ml.azure.com/runs/HD_0527fb3f-98be-4473-9bc3-32fdd153d953?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-154263/workspaces/quick-starts-ws-154263&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-11T12:25:32.639644][API][INFO]Experiment created<END>\n""<START>[2021-08-11T12:25:33.571561][GENERATOR][INFO]Trying to sample '10' jobs from the hyperparameter space<END>\n""<START>[2021-08-11T12:25:33.786642][GENERATOR][INFO]Successfully sampled '10' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_0527fb3f-98be-4473-9bc3-32fdd153d953
Web View: https://ml.azure.com/runs/HD_0527fb3f-98be-4473-9bc3-32fdd153d953?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-154263/workspaces/quick-starts-ws-154263&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_0527fb3f-98be-4473-9bc3-32fdd153d953',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-11T12:25:32.389645Z',
 'endTimeUtc': '2021-08-11T12:33:37.558532Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '3b82b012-13bc-4019-a39b-97226c952a27',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.32.0',
  'score': '0.908649468892261',
  'best_child_run_id': 'HD_0527fb3f-98be-4473-9bc3-32fdd153d953_4',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg154263.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_0527fb3f-98be-4473-9bc3-32fdd153d953/azureml-logs/hyperdrive.txt?sv=2019-07-07&sr=b&si

In [10]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_file_names())
hyperdrive_model = best_run.register_model(model_name='best-model', model_path = 'outputs/model.joblib')
#print(best_run)


['azureml-logs/55_azureml-execution-tvmps_684753454b8fd0b9980244084905eff630a96716a8deaa277801e9c70ede8afa_d.txt', 'azureml-logs/65_job_prep-tvmps_684753454b8fd0b9980244084905eff630a96716a8deaa277801e9c70ede8afa_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_684753454b8fd0b9980244084905eff630a96716a8deaa277801e9c70ede8afa_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/105_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']
Run(Experiment: udacity-project-1,
Id: HD_0527fb3f-98be-4473-9bc3-32fdd153d953_4,
Type: azureml.scriptrun,
Status: Completed)


In [36]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset 

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
ds = TabularDatasetFactory.from_delimited_files(url)

In [37]:
from train import clean_data
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=66)
x_train.loc[:,'y'] = y_train.values
x_test.loc[:,'y'] = y_test.values
x_train.to_csv("./training/train_data.csv",index=False)
x_test.to_csv("test_data.csv",index=False)

# registering data in azure

dataset = ws.get_default_datastore()
#print(dataset)
dataset.upload(src_dir='./training', target_path='udacity-project-1', overwrite=True, show_progress=True)
train_data = Dataset.Tabular.from_delimited_files(path=dataset.path('udacity-project-1/train_data.csv'))



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Uploading an estimated of 1 files
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [38]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=3,
    compute_target=cpu_cluster,
    enable_onnx_compatible_models=True)

In [ ]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output=True)
RunDetails(remote_run).show()

Submitting remote run.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project-1,AutoML_c3bb2cc9-3f3f-4e35-ad98-b65df9e0481a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

In [ ]:
# Retrieve and save your best automl model.

best_automl_run, model = remote_run.get_output()
print(best_automl_run)
best_automl_run.register_model(model_name='automl.pkl', model_path='./outputs')
print(model._final_estimator)